# Too slow if read part file, waiting for better solution.

In [1]:
import os
import SimpleITK as sitk
import numpy as np
import time

In [3]:
start_t = time.time()
count = 0
for i in range(1,16):
    count+=1
    patch_ct_path = r"E:\Script_hist\patch\com30\hist00\ctimg0001\image{}.mha".format(i)
    patch_image=sitk.ReadImage(patch_ct_path)
    patch_image_arry = sitk.GetArrayFromImage(patch_image)
end16_t = time.time()
print(end16_t-start_t)
print(count)

0.09618020057678223
15


In [4]:
patch_image_arry.shape

(128, 128, 32)

In [7]:
patch_size = (128,128,32)
start_point = (128,128,32)
file_shape = (512,512,147)

def read_patch(patch_size,start_point,file_shape,file):
    pic = []
    with open(file,"rb") as f:
        for xi in range(start_point[0],start_point[0]+patch_size[0]):
            for yi in range(start_point[1],start_point[1]+patch_size[1]):
                start = xi*file_shape[1]*file_shape[2]+yi*file_shape[2]+start_point[2]
                f.seek(start)
                nums = f.readlines(patch_size[2])
                
                for i in range(0,patch_size[2]*2,2):
                    num = int.from_bytes(nums[i:i+2],byteorder='little', signed=True)
                    pic.append(num)
    return np.array(pic).reshape(patch_size)


In [8]:
start_t = time.time()
count = 0
for i in range(0,255,16):
    start_point = (i,i,32)
    patch_image_arry = read_patch(patch_size,start_point,file_shape,"speed_test.txt")
    count+=1
end16_t = time.time()
print(end16_t-start_t)
print(count)

4.659423351287842
16


In [2]:
example_ct_path = r"E:\hist_01\comp30\img0001\c_ct.mha"
image=sitk.ReadImage(example_ct_path)
image_arry = sitk.GetArrayFromImage(image)

In [19]:
# x,y,z
list_ct = list(image_arry.flatten())
for i in range(len(list_ct)):
    list_ct[i] = str(list_ct[i])

In [17]:
list_ct_int = list(image_arry.flatten())

In [20]:
with open("speed_test_int.txt","wb") as f:
    f.writelines(list_ct_int)

In [20]:
with open("speed_test.txt","w") as f:
    f.writelines([line+'\n' for line in list_ct])

In [42]:
print(list_ct_int[0].dtype)
print(len(list_ct_int))

int16
38535168


In [21]:
with open("speed_test.txt","r") as fin:
    #fin.seek(0)
    data = fin.readlines()
len(data)

38535168

In [18]:
data

'-1024-1024'

In [52]:
pic = []

for i in range(0,1000,2):
    num = int.from_bytes(data[i:i+2],byteorder='little', signed=True)
    pic.append(num)

In [90]:
result = read_patch(patch_size,start_point,file_shape,"speed_test_int.txt")